In [ ]:
!pip install git+https://github.com/openai/jukebox.git

IMPORTANT NOTE ON SYSTEM REQUIREMENTS:

If you are connecting to a hosted runtime, make sure it has a P100 GPU (optionally run !nvidia-smi to confirm). Go to Edit>Notebook Settings to set this.

CoLab may first assign you a lower memory machine if you are using a hosted runtime.  If so, the first time you try to load the 5B model, it will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model.

If you are using a local GPU, we recommend V100 or P100 with 16GB GPU memory for best performance. For GPU’s with less memory, we recommend using the 1B model and a smaller batch size throughout.  

システム要件に関する重要な注意事項。

ホストランタイムに接続している場合、P100 GPUを搭載していることを確認してください(オプションで !nvidia-smi を実行して確認してください)。編集] > [ノートブックの設定]で設定してください。

CoLabは、ホストランタイムを使用している場合、最初に低いメモリマシンを割り当てることがあります。その場合、最初に5Bモデルをロードしようとすると、メモリが不足してしまい、より多くのメモリで再起動するように促されます（その後、このCoLabのトップに戻ります）。この後もメモリの問題が続く場合（または自宅のセットアップで問題が発生した場合）は、1Bモデルに切り替えてください。

ローカルGPUを使用している場合は、最高のパフォーマンスを得るために16GBのGPUメモリを搭載したV100またはP100をお勧めします。メモリが少ないGPUの場合は、1Bモデルを使用し、全体的に小さいバッチサイズを使用することをお勧めします。

In [ ]:
!nvidia-smi

In [ ]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

# Sample from the 5B or 1B Lyrics Model
5Bまたは1Bの歌詞モデルからのサンプル


In [ ]:
model = "5b_lyrics" # or "1b_lyrics"     
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model=='5b_lyrics' else 8
hps.name = 'samples'
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)



Specify your choice of artist, genre, lyrics, and length of musical sample. <br>
アーティスト、ジャンル、歌詞、音楽サンプルの長さを指定してください。

生成する音楽サンプルの完全な長さ - サンプルの長さに比例した生成時間で、1分から4分の範囲の曲がよく機能することがわかります。 

この合計長さは、モデルがどれだけ早く歌詞を進行するかに影響します（モデルはまた、それがサンプルの最初、真ん中、または終わりにあると考えるかどうかによって、異なる生成を行います）。

In [ ]:
sample_length_in_seconds = 60          # Full length of musical sample to generate - we find songs in the 1 to 4 minute
                                       # range work well, with generation time proportional to sample length.  
                                       # This total length affects how quickly the model 
                                       # progresses through lyrics (model also generates differently
                                       # depending on if it thinks it's in the beginning, middle, or end of sample)

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [ ]:
metas = [dict(artist = "Zac Brown Band",
            genre = "Country",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """I met a traveller from an antique land,
            Who said—“Two vast and trunkless legs of stone
            Stand in the desert. . . . Near them, on the sand,
            Half sunk a shattered visage lies, whose frown,
            And wrinkled lip, and sneer of cold command,
            Tell that its sculptor well those passions read
            Which yet survive, stamped on these lifeless things,
            The hand that mocked them, and the heart that fed;
            And on the pedestal, these words appear:
            My name is Ozymandias, King of Kings;
            Look on my Works, ye Mighty, and despair!
            Nothing beside remains. Round the decay
            Of that colossal Wreck, boundless and bare
            The lone and level sands stretch far away
            """,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

Optionally adjust the sampling temperature (we've found .98 or .99 to be our favorite).  <br>
オプションでサンプリング温度を調整することができます（0.98または0.99がお気に入りです）。

In [ ]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model == "5b_lyrics" else 16
lower_level_chunk_size = 32
chunk_size = 16 if model == "5b_lyrics" else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

Now we're ready to sample from the model. We'll generate the top level (2) first, followed by the first upsampling (level 1), and the second upsampling (0).  In this CoLab we load the top prior separately from the upsamplers, because of memory concerns on the hosted runtimes. If you are using a local machine, you can also load all models directly with make_models, and then use sample.py's ancestral_sampling to put this all in one step.

After each level, we decode to raw audio and save the audio files.   

This next cell will take a while (approximately 10 minutes per 20 seconds of music sample)

これで、モデルからサンプリングする準備ができました。<br>
最初にトップレベル(2)を生成し、その後に最初のアップサンプリング(レベル1)、2番目のアップサンプリング(0)を行います。<br>
このCoLabでは、ホストされたランタイムではメモリの問題があるため、アップサンプラーとは別にトップの先行処理をロードします。<br>
ローカルマシンを使用している場合は、make_modelsを使用して直接すべてのモデルをロードし、sample.pyのancestral_samplingを使用して1つのステップですべてのモデルをロードすることもできます。<br>

各レベルの後、生のオーディオにデコードして、オーディオファイルを保存します。<br>

この次のセルは時間がかかります（音楽サンプル20秒あたり約10分）。<br>

In [ ]:
zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)

Listen to the results from the top level (note this will sound very noisy until we do the upsampling stage).  You may have more generated samples, depending on the batch size you requested.

トップレベルの結果を聞いてください（アップサンプリングを行うまでは、非常にノイズの多い音になることに注意してください）。要求されたバッチサイズに応じて、より多くのサンプルが生成されるかもしれません。

In [ ]:
Audio(f'{hps.name}/level_2/item_0.wav')

We are now done with the large top_prior model, and instead load the upsamplers.<br>
これで大規模なtop_priorモデルの処理は終了し、代わりにアップサンプラをロードします。

十分なメモリがあるローカルマシンを使用している場合は、このFalseを設定してください<br>（これにより、アップサンプリングの段階で歌詞のアラインメントの可視化を行うことができます）。<br>
ホスト型ランタイムでは、5b_lyricsモデルを使用している場合、top_priorを削除する必要があります。

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

Please note: this next upsampling step will take several hours.  At the free tier, Google CoLab lets you run for 12 hours.  As the upsampling is completed, samples will appear in the Files tab (you can access this at the left of the CoLab), under "samples" (or whatever hps.name is currently).  Level 1 is the partially upsampled version, and then Level 0 is fully completed.

注意：この次のアップサンプリングのステップには数時間かかります。<br>
Google CoLab の無料版では、12時間の実行が可能です。<br>
アップサンプリングが完了すると、サンプルが Files タブ（CoLab の左側にあります）の "samples"（または現在の hps.name が何であれ）の下に表示されます。<br>
レベル1が部分的にアップサンプリングされたバージョンで、レベル0が完全に完了した状態です。<br>

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

Listen to your final sample!

In [ ]:
Audio(f'{hps.name}/level_0/item_0.wav')

In [ ]:
del upsamplers
empty_cache()

# Co-Composing with the 5B or 1B Lyrics Model
5Bや1Bの作詞モデルとの共作曲

For more control over the generations, try co-composing with either the 5B or 1B Lyrics Models.  Again, specify your artist, genre, and lyrics. However, now instead of generating the entire sample, the model will return 3 short options for the opening of the piece (or up to 16 options if you use the 1B model instead).  Choose your favorite, and then continue the loop, for as long as you like.  Throughout these steps, you'll be listening to the audio at the top prior level, which means it will sound quite noisy.  When you are satisfied with your co-creation, continue on through the upsampling section. This will render the piece in higher audio quality.

NOTE: CoLab will first assign you a lower memory machine if you are using a hosted runtime.  The next cell will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model. 

世代をよりコントロールしたい場合は、5Bまたは1B歌詞モデルとの共作を試してみてください。ここでも、アーティスト、ジャンル、歌詞を指定します。しかし、サンプル全体を生成するのではなく、このモデルは、曲のオープニングに3つの短いオプションを返します（代わりに1Bモデルを使用した場合は、最大16のオプションを返します）。好きなものを選んで、好きなだけループを続けてください。これらのステップの間、あなたはトップの前のレベルでオーディオを聞くことになるので、それはかなりノイズの多い音になります。共作に満足したら、アップサンプリングセクションに進みます。これにより、より高いオーディオ品質で作品をレンダリングすることができます。

注：CoLab は、ホストランタイムを使用している場合、最初に低いメモリマシンを割り当てます。次のセルはメモリを使い果たした後、より多くのメモリで再起動するように促されます（その後、この CoLab のトップに戻ります）。この後もメモリの問題が続く場合（またはホームセットアップで問題が発生した場合）は、1Bモデルに切り替えてください。

In [ ]:
model = "5b_lyrics" # or "1b_lyrics"
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model=='5b_lyrics' else 16
hps.name = 'co_composer'
hps.sample_length = 1048576 if model=="5b_lyrics" else 786432 
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.hop_fraction = [.5, .5, .125] 
hps.levels = 3

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = hps.sample_length)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

Choose your artist, genre, and lyrics here!

アーティスト、ジャンル、歌詞を選ぶならここ!

In [ ]:
total_sample_length_in_seconds = 120
metas = [dict(artist = "Zac Brown Band",
            genre = "Country",
            total_length = total_sample_length_in_seconds * hps.sr,
            offset = 0,
            lyrics = """I met a traveller from an antique land,
            Who said—“Two vast and trunkless legs of stone
            Stand in the desert. . . . Near them, on the sand,
            Half sunk a shattered visage lies, whose frown,
            And wrinkled lip, and sneer of cold command,
            Tell that its sculptor well those passions read
            Which yet survive, stamped on these lifeless things,
            The hand that mocked them, and the heart that fed;
            And on the pedestal, these words appear:
            My name is Ozymandias, King of Kings;
            Look on my Works, ye Mighty, and despair!
            Nothing beside remains. Round the decay
            Of that colossal Wreck, boundless and bare
            The lone and level sands stretch far away
            """,
            ),
          ] * hps.n_samples
labels = top_prior.labeller.get_batch_labels(metas, 'cuda')

## Generate 3 options for the start of the song
曲の開始時に3つのオプションを生成する

Initial generation is set to be 4 seconds long, but feel free to change this<br>
初期生成は4秒に設定されていますが、ご自由に変更してください。

In [ ]:
def seconds_to_tokens(sec, sr, prior, chunk_size):
  tokens = sec * hps.sr // prior.raw_to_tokens
  tokens = ((tokens // chunk_size) + 1) * chunk_size
  assert tokens <= prior.n_ctx, 'Choose a shorter generation length to stay within the top prior context'
  return tokens

In [ ]:
initial_generation_in_seconds = 4
tokens_to_sample = seconds_to_tokens(initial_generation_in_seconds, hps.sr, top_prior, chunk_size)

Change the sampling temperature if you like (higher is more random).  Our favorite is in the range .98 to .995

お好みでサンプリング温度を変更してください（高い方がよりランダムです）。私たちのお気に入りは0.98から0.995の範囲です。

In [ ]:
sampling_temperature = .98
sampling_kwargs = dict(temp=sampling_temperature, fp16=True,
                       max_batch_size=max_batch_size, chunk_size=chunk_size)

In [ ]:
zs=[t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(3)]
zs=sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

Listen to your generated samples, and then pick a favorite. If you don't like any, go back and rerun the cell above. 

生成されたサンプルを聞いて、気に入ったものを選んでください。気に入らないものがあれば、上のセルに戻って再実行してください。

** NOTE this is at the noisy top level, upsample fully (in the next section) to hear the final audio version

** 注意：これはノイズの多いトップレベルで、最終的なオーディオバージョンを聞くには（次のセクションで）完全にアップサンプルしてください。

In [ ]:
for i in range(hps.n_samples):
  librosa.output.write_wav(f'noisy_top_level_generation_{i}.wav', x[i], sr=44100)

In [ ]:
Audio('noisy_top_level_generation_0.wav')

In [ ]:
Audio('noisy_top_level_generation_1.wav')

In [ ]:
Audio('noisy_top_level_generation_2.wav')

If you don't like any of the options, return a few cells back to "Sample a few options..." and rerun from there.

## Choose your favorite sample and request longer generation

---

(Repeat from here)


In [ ]:
my_choice=0

In [ ]:
zs[2]=zs[2][my_choice].repeat(hps.n_samples,1)
t.save(zs, 'zs-checkpoint2.t')

In [ ]:
# Set to True to load the previous checkpoint:
if False:
  zs=t.load('zs-checkpoint2.t') 

Choose the length of the continuation.  The 1B model can generate up to 17 second samples and the 5B up to 23 seconds, but you'll want to pick a shorter continuation length so that it will be able to look back at what you've generated already.  Here we've chosen 4 seconds.

どの選択肢も気に入らない場合は、「いくつかの選択肢のサンプルを...」とセルを戻して、そこから再実行します。

In [ ]:
continue_generation_in_seconds=4
tokens_to_sample = seconds_to_tokens(continue_generation_in_seconds, hps.sr, top_prior, chunk_size)

The next step asks the top prior to generate more of the sample. It'll take up to a few minutes, depending on the sample length you request.

次のステップでは、サンプルをさらに生成するようにトップの前に依頼します。要求するサンプルの長さにもよりますが、最大で数分かかります。

In [ ]:
zs = sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

Now listen to the longer versions of the sample you selected, and again choose a favorite sample.  If you don't like any, return back to the cell where you can load the checkpoint, and continue again from there.

When the samples start getting long, you might not always want to listen from the start, so change the playback start time later on if you like.

ここで、選択したサンプルの長いバージョンを聴いて、もう一度好きなサンプルを選びます。気に入らないものがあれば、チェックポイントを読み込めるセルに戻り、そこからもう一度続けます。

サンプルが長くなってきたら、最初から聴きたいとは限らないので、再生開始時間を後から変更してみてください。

In [ ]:
playback_start_time_in_seconds = 0 

In [ ]:
for i in range(hps.n_samples):
  librosa.output.write_wav(f'top_level_continuation_{i}.wav', x[i][playback_start_time_in_seconds*44100:], sr=44100)

In [ ]:
Audio('top_level_continuation_0.wav')

In [ ]:
Audio('top_level_continuation_1.wav')

In [ ]:
Audio('top_level_continuation_2.wav')

To make a longer song, return back to "Choose your favorite sample" and loop through that again

より長い曲を作るには、"Choose your favorite sample "に戻って、もう一度ループします。

# Upsample Co-Composition to Higher Audio Quality
アップサンプル合成による高音質化

Choose your favorite sample from your latest group of generations.  (If you haven't already gone through the Co-Composition block, make sure to do that first so you have a generation to upsample).

最新の世代グループからお気に入りのサンプルを選択します。(まだCo-Compositionブロックを通過していない場合は、アップサンプルする世代があるので、最初にそれを行うようにしてください)。

In [ ]:
choice = 0
select_best_sample = True  # Set false if you want to upsample all your samples 
                           # upsampling sometimes yields subtly different results on multiple runs,
                           # so this way you can choose your favorite upsampling

In [ ]:
if select_best_sample:
  zs[2]=zs[2][choice].repeat(zs[2].shape[0],1)

t.save(zs, 'zs-top-level-final.t')

Note: If you are using a CoLab hosted runtime on the free tier, you may want to download this zs-top-level-final.t file, and then restart an instance and load it in the next cell.  The free tier will last a maximum of 12 hours, and the upsampling stage can take many hours, depending on how long a sample you have generated.

注意：free tierでCoLabホストランタイムを使用している場合は、この "zs-top-level-final.t "ファイルをダウンロードし、インスタンスを再起動して次のセルにロードするとよいでしょう。無料層では最大12時間、アップサンプリングの段階では、生成したサンプルの長さに応じて何時間もかかることがあります。

In [ ]:
if False:
  zs = t.load('zs-top-level-final.t')

assert zs[2].shape[1]>=2048, f'Please first generate at least 2048 tokens at the top level, currently you have {zs[2].shape[1]}'
hps.sample_length = zs[2].shape[1]*top_prior.raw_to_tokens

十分なメモリを持つローカルマシン上にいる場合は、これをFalseに設定してください<br>(これにより歌詞のアラインメントの可視化を行うことができます)。<br>
ホスト型ランタイムでは、5b_lyricsモデルを使用している場合、top_priorを削除する必要があります。

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization). For a hosted runtime, we'll need to go ahead and delete the top_prior
# if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None

upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]

In [ ]:
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=16, chunk_size=32),
                    dict(temp=0.99, fp16=True, max_batch_size=16, chunk_size=32),
                    None]

if type(labels)==dict:
  labels = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers] + [labels] 

This next step upsamples 2 levels.  The level_1 samples will be available after around one hour (depending on the length of your sample) and are saved under {hps.name}/level_0/item_0.wav, while the fully upsampled level_0 will likely take 4-12 hours. You can access the wav files down below, or using the "Files" panel at the left of this CoLab.

この次のステップでは、2 つのレベルをアップサンプリングします。level_1 のサンプルは約 1 時間後に利用可能になり（サンプルの長さにもよります）、{hps.name}/level_0/item_0.wav に保存されます。wav ファイルにアクセスするには、下の方、またはこの CoLab の左側にある "Files" パネルを使用してください。

(Please note, if you are using this CoLab on Google's free tier, you may want to download intermediate steps as the connection will last for a maximum 12 hours.)

(Google の無料版でこの CoLab を使用している場合は、接続が最大 12 時間持続するため、中間的なステップをダウンロードすることをお勧めします)

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

In [ ]:
Audio(f'{hps.name}/level_0/item_0.wav')